<a href="https://colab.research.google.com/github/huongtravo0910/pdf-question-answer-rag-llm/blob/feature%2Fcreate_question_answer_pdf_app_with_rag/pdf_question_answer_rag_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Download dataset

In [2]:
# https://drive.google.com/file/d/1LwGdnwxEnU9WDsDTDQ3Z9N45f__nZwls/view?usp=sharing
!gdown 1LwGdnwxEnU9WDsDTDQ3Z9N45f__nZwls

Downloading...
From: https://drive.google.com/uc?id=1LwGdnwxEnU9WDsDTDQ3Z9N45f__nZwls
To: /content/ngu-hanh.pdf.crdownload
100% 6.44M/6.44M [00:00<00:00, 33.0MB/s]


#2. Install and import library

In [3]:
!pip install -q transformers==4.41.2
!pip install -q bitsandbytes==0.43.1
!pip install -q accelerate==0.31.0
!pip install -q langchain==0.2.5
!pip install -q langchainhub==0.1.20
!pip install -q langchain-chroma==0.1.1
!pip install -q langchain-community==0.2.5
!pip install -q langchain_huggingface==0.0.3
!pip install -q python-dotenv==1.0.1
!pip install -q pypdf==4.2.0
!pip install -q numpy==1.24.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [4]:
import torch

from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface.llms import HuggingFacePipeline

from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.chains import ConversationalRetrievalChain

from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub

#3. Read PDF File

In [5]:
Loader = PyPDFLoader
FILE_PATH = "./ngu-hanh.pdf.crdownload"
loader = Loader(FILE_PATH)
documents = loader.load()

print("Number of documents: ", len(documents))
documents[0]

Number of documents:  38


Document(metadata={'source': './ngu-hanh.pdf.crdownload', 'page': 0}, page_content=' 1 \n  Tiết 2 Ngũ Hành TỨ ĐẠI & NGŨ HÀNH   Nguyễn Quốc Bảo    Thou hast as chiding a nativity As fire, water, earth and heaven can make To herald thee from the womb PERICLES, Shakespeare, Pericles Prince of Tyre.  CHÂN LÝ TOÀN NĂNG Tiết 1 : Nguyên lý Nhị nguyên Tiết 2 : Ngũ hành Tiết 3 : Bát quái Titre 4 : Hệ số Titre 5 : Hoàng đạo Titre 6 : Thượng Đế hình học  Trong bài Nguyên lí Nhị Nguyên tiết 1, có đoạn viết:  ... Hệ Từ Thượng Truyện viết nguồn gốc của Vũ Trụ: Vô Cực sinh Thái Cực, Thái Cực sinh Luỡng Nghi, Lưỡng Nghi sinh Tứ Tượng Ngũ Hành, Ngũ Hành sinh Bát Quái, Bát Quái là gốc của 64 quẻ Kinh dịch. ... Thành thử Âm Dương đi 2 lối khác nhau, Tam Tài Ngũ Hành (Dương, với Số lẻ) và Tứ Tượng Bát Quái (Âm, với Số chẵn).  ... Âm Dương chuyển hóa theo chu kỳ Ngũ Hành Kim Mộc Thủy Hỏa Thổ, để quy định những quy luật tự nhiên trong xã hội loài người.  HỆ TỪ THƯỢNG HẠ TRUYỆN là 2 quyển nằm trong bộ Thập d

#4. Initialize text splitter and split document

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               chunk_overlap=100)

In [7]:
docs = text_splitter.split_documents(documents)

print("Number of mini-documents: ", len(docs))
docs[100]

Number of mini-documents:  122


Document(metadata={'source': './ngu-hanh.pdf.crdownload', 'page': 29}, page_content='Vệ, nhưng sau lại mất ở nước Tề. Đời sau phong ông là Lê Dương Hầu. Tử Hạ hỏi: Tử Cống là người thế nào? Đức Khổng Tử đáp: Cái nhanh của Tứ hơn ta (Khổng Tử Gia Ngữ).  (21) Đức Khổng Tử tên là Khâu, tự là Trọng Ni, sanh ngày 27 tháng 8 âm lịch năm Canh Tuất (551 trước Tây lịch), đời vua Châu Linh Vương năm thứ 21 nhà Châu, tương ứng với đời vua Lỗ Tương Công năm thứ 22, tại làng Xương Bình, huyện Khúc Phụ, nước Lỗ, bây giờ là Tỉnh Sơn Đông nước Trung Hoa.  (22) Đây chỉ là dịch thoát ý bản chữ Pháp, vì không có nguyên bản chữ Hán của sách Lễ Ký về đoạn này.  (23) Khổng An Quốc nhà Hán nói: “Hà đồ khi Họ Phục Hy làm Vua thiên hạ, có con Long mã hiện ở sông Hà, bèn bắt chước cái vằn của nó vạch ra tám Quẻ. Lạc thư là khi Vua Vũ chữa được nước Lụt, có con Rùa thần đội vằn, trên lưng có số tới chín, nhân đó xếp Thứ tự ra chín Loài”. Lưu Hâm nhà Hán cũng nói: “Họ Phục Hy nối Trời làm Vua, nhận Đồ sông Hà mà 

#5. Initialize text vectorization (embedding)

In [8]:
embedding = HuggingFaceEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#6. Create vector database and retriever

In [9]:
vector_db = Chroma.from_documents(documents=docs,
                                  embedding=embedding)

retriever = vector_db.as_retriever()

QUERY = "Hành Thuỷ là gì?"
result = retriever.invoke(QUERY)

print("Number of relevant documents: ", len(result))
result[0]

Number of relevant documents:  4


Document(metadata={'page': 6, 'source': './ngu-hanh.pdf.crdownload'}, page_content='dụng dễ hiểu là Máy hơi nước, nếu không có Nồi hơi (Kim), Thủy sẽ làm tắt Hỏa. Nồi hơi trung gian Kim đã để Hỏa biến Thủy thành hơi nước. Bây giờ, thử xem xét từng Nguyên tố của Ngũ Hành.  Tương quan giữa các Nguyên tố Ngũ Hành và các Quẻ Kinh Dịch :  Nguyên tố  Kinh Dịch  Quẻ  Ký Hiệu - Hán tự  Mộc  Gió, Sét  Tốn - Chấn  :||(☴巽xùntốn)|::(☳震zhènchấn )  Hỏa  Lửa  Li  |:| (☲ 離 lí li)')

#7. Load LLMS(Vicuna)

In [10]:
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

MODEL_NAME = "lmsys/vicuna-7b-v1.5"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=nf4_config,
    low_cpu_mem_usage=True
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    pad_token_id=tokenizer.eos_token_id,
    device_map="auto"
)

llm = HuggingFacePipeline(
    pipeline=model_pipeline,
)

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

#8. Prompting with RAG

In [11]:
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

USER_QUESTION = "Hành Thủy là gì?"
output = rag_chain.invoke(USER_QUESTION)
output

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:520: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


"Human: You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: Hành Thủy là gì? \nContext: dụng dễ hiểu là Máy hơi nước, nếu không có Nồi hơi (Kim), Thủy sẽ làm tắt Hỏa. Nồi hơi trung gian Kim đã để Hỏa biến Thủy thành hơi nước. Bây giờ, thử xem xét từng Nguyên tố của Ngũ Hành.  Tương quan giữa các Nguyên tố Ngũ Hành và các Quẻ Kinh Dịch :  Nguyên tố  Kinh Dịch  Quẻ  Ký Hiệu - Hán tự  Mộc  Gió, Sét  Tốn - Chấn  :||(☴巽xùntốn)|::(☳震zhènchấn )  Hỏa  Lửa  Li  |:| (☲ 離 lí li)\n\nNam qua sông Hoàng hà, gặp Long mã, trên lưng có những chấm đen trắng, rồi thấy rùa thần hiện ở sông Lạc, trên lưng cũng có trang chữ. Phục Hi dựa vào đó vẽ lại thành Hà đồ và Lạc thư, rồi ngửa đầu xem tượng trời, cúi xuống xem thế đất, mà vẽ nên Bát quái (Xem Đũa, Tiết 1). Hà đồ chép lại đời Thương đã ghi rõ ý niệm Ngũ Hành 

In [12]:
answer = output.split('Answer:')[1].strip()
answer

'Hành Thủy là một trong năm hành của Ngũ Hành, tương ứng với hành Bắc. Nó được định nghĩa là một hành có tính năng thủy triều, tức là có khả năng thay đổi và chuyển đổi giữa các hành khác nhau. Hành Thủy tương tác với hành Hỏa ở hành Nam, và cả hai hành có khả năng tương tác với nhau để tạo ra năng lượng và sự thay đổi trong tự nhiên.'